In [ ]:
# from models.sos import ScalarOnScalarModel
# from optimizers.nbdo import NBDO
# from bases.bspline import BSplineBasis
# from models.sof import ScalarOnFunctionModel
# import numpy as np

In [ ]:
# # SoS
# model = ScalarOnScalarModel(Kx=3, criterion="A", order=2)
# nbdo = NBDO(model=model, latent_dim=10, seed=42)
# nbdo.compute_train_set(num_designs=1_000, runs=25)
# nbdo.fit(epochs=1_000, patience=100, batch_size=256)
# crit, design = nbdo.optimize(n_calls=30)
# np.round(crit,2)

In [ ]:
# # SoF
# xB, bB = BSplineBasis(0,4), BSplineBasis(0,2)
# model = ScalarOnFunctionModel([(xB,bB)], criterion="D", intercept=True)
# nbdo = NBDO(model=model, latent_dim=2, seed=42, verbose=False)
# nbdo.compute_train_set(num_designs=1_000, runs=12)
# nbdo.fit(epochs=10, patience=5, batch_size=256)
# crit, design = nbdo.optimize(n_calls=5)
# crit

In [ ]:
# from diagnostics import (
#     info_matrix, eigen_spectrum, condition_number,
#     leverage_diag, prediction_variance,
# )

# M = info_matrix(model, design)
# eigvals, _ = eigen_spectrum(M)
# kappa = condition_number(M)
# print(f"p = {M.shape[0]}, λ_min = {eigvals[-1]:.3e}, λ_max = {eigvals[0]:.3e}, κ2 = {kappa:.2e}")

# h = leverage_diag(model, design)
# print(f"leverage stats: min={h.min():.3f}, max={h.max():.3f}, mean={h.mean():.3f}")

In [ ]:
# p = 10, λ_min = 3.458e+00, λ_max = 5.728e+01, κ2 = 1.66e+01
# leverage stats: min=0.179, max=0.748, mean=0.400

In [ ]:
# # quick smoke test
# from bases.bspline import BSplineBasis
# from models.fof import FunctionOnFunctionModel
# from optimizers.nbdo import NBDO

# def B(K, deg):
#     return BSplineBasis(degree=deg, total_knots_num=max(2, K - deg + 1))

# bx, bb = B(5,2), B(6,2)
# model = FunctionOnFunctionModel([(bx, bb)], criterion="A", intercept=True)
# opt = NBDO(model=model, latent_dim=4, seed=0)
# opt.compute_train_set(num_designs=1_000, runs=12)
# opt.fit(epochs=1_000, patience=100, batch_size=256)       # should converge & print losses
# report, design = opt.optimize(n_calls=5, n_random_starts=3)
# print("ok ✅", report, design.shape)

In [ ]:
# import numpy as np
# from bases.bspline import BSplineBasis
# from models.sof import ScalarOnFunctionModel
# from models.fof import FunctionOnFunctionModel

# def B(K, deg):
#     total_knots_num = max(2, K - deg + 1)
#     return BSplineBasis(degree=deg, total_knots_num=total_knots_num)

# # Two predictors, different dims
# bx1, bb1 = B(5,2), B(6,2)
# bx2, bb2 = B(4,3), B(3,3)
# pairs = [(bx1, bb1), (bx2, bb2)]

# runs = 14

# fof = FunctionOnFunctionModel(pairs, criterion="A", intercept=True)
# sof = ScalarOnFunctionModel(pairs, criterion="A", intercept=True)

# # Build Γ by concatenating per-predictor blocks
# rng = np.random.default_rng(0)
# Gamma = np.hstack([
#     rng.normal(size=(runs, fof.Kx_list[0])),
#     rng.normal(size=(runs, fof.Kx_list[1])),
# ])

# # Block-diag sanity: each slice maps correctly
# Phi = Gamma @ fof.J_np
# for xs, bs in zip(fof.x_slices, fof.b_slices):
#     Ji = fof.J_np[xs, :][:, bs]
#     assert np.allclose(Gamma[:, xs] @ Ji, Phi[:, bs], atol=1e-12)

# # Shapes and parity
# Zf, Zs = fof.model_matrix(Gamma), sof.model_matrix(Gamma)
# Mf, Ms = fof.information_matrix(Zf), sof.information_matrix(Zs)
# print("Z shapes:", Zf.shape, Zs.shape)
# print("M shapes:", Mf.shape, Ms.shape)

# vf, vs = fof.objective_num(Gamma), sof.objective_num(Gamma)
# print("A-opt FoF vs SoF:", vf, vs)
# assert np.allclose(vf, vs, rtol=1e-10, atol=1e-10)

# print("Multi-predictor behavior matches SoF ✅")

In [ ]:
# # --- Scalar-on-Function parity baseline ---

# import numpy as np
# import tensorflow as tf

# from bases.bspline import BSplineBasis
# from models.sof import ScalarOnFunctionModel

# # reproducibility
# rng = np.random.default_rng(12345)
# tf.keras.utils.set_random_seed(12345)

# def B(K: int, degree: int) -> BSplineBasis:
#     """
#     Your BSplineBasis signature: (degree, total_knots_num)
#     Relation: K = degree + total_knots_num - 1  -> total_knots_num = K - degree + 1
#     """
#     total_knots_num = max(2, K - degree + 1)
#     return BSplineBasis(degree=degree, total_knots_num=total_knots_num)

# # --- common design setup (multi-predictor) ---
# deg1, deg2 = 2, 2
# Kx1, Kb1 = 5, 6    # predictor 1: design basis size, beta(s,·) s-basis size
# Kx2, Kb2 = 4, 3    # predictor 2: design basis size, beta(s,·) s-basis size

# bx1, bb1 = B(Kx1, deg1), B(Kb1, deg1)
# bx2, bb2 = B(Kx2, deg2), B(Kb2, deg2)
# pairs = [(bx1, bb1), (bx2, bb2)]

# intercept = True
# Kb_total = Kb1 + Kb2
# p = Kb_total + (1 if intercept else 0)

# runs = 14  # ensure runs > p for a well-conditioned M

# # --- build model(s) ---
# sof_A = ScalarOnFunctionModel(pairs, criterion="A", intercept=intercept, dtype=tf.float64)
# sof_D = ScalarOnFunctionModel(pairs, criterion="D", intercept=intercept, dtype=tf.float64)

# # --- design coefficients Γ: (runs, Kx_total) ---
# Gamma = rng.normal(size=(runs, sof_A.Kx))

# # --- shapes & quick info ---
# print("=== Scalar-on-Function (baseline) ===")
# print(f"Kx_list={sof_A.Kx_list}, sum={sof_A.Kx} | Kb_list={sof_A.Kb_list}, sum={sof_A.Kb}")
# print(f"p={sof_A.p} (should equal {p}), runs={runs}")
# print("J shape:", sof_A.J_np.shape)

# Z = sof_A.model_matrix(Gamma)
# M = sof_A.information_matrix(Z)
# print("Z shape:", Z.shape, "| M shape:", M.shape)

# # --- criteria values ---
# A_val = sof_A.objective_num(Gamma)
# D_val = sof_D.objective_num(Gamma)

# print("\nResults to copy:")
# print(f"A-opt (trace(M^{-1})) loss: {A_val:.12f}")
# print(f"D-opt (-logdet M) loss:     {D_val:.12f}")


In [ ]:
# # --- Function-on-Function parity check (should match SoF numbers) ---

# import numpy as np
# import tensorflow as tf

# from bases.bspline import BSplineBasis
# from models.fof import FunctionOnFunctionModel

# # reproducibility (same seeds as in Cell 1)
# rng = np.random.default_rng(12345)
# tf.keras.utils.set_random_seed(12345)

# def B(K: int, degree: int) -> BSplineBasis:
#     total_knots_num = max(2, K - degree + 1)
#     return BSplineBasis(degree=degree, total_knots_num=total_knots_num)

# # --- same common design setup (multi-predictor) ---
# deg1, deg2 = 2, 2
# Kx1, Kb1 = 5, 6
# Kx2, Kb2 = 4, 3

# bx1, bb1 = B(Kx1, deg1), B(Kb1, deg1)
# bx2, bb2 = B(Kx2, deg2), B(Kb2, deg2)
# pairs = [(bx1, bb1), (bx2, bb2)]

# intercept = True
# Kb_total = Kb1 + Kb2
# p = Kb_total + (1 if intercept else 0)

# runs = 14  # same as Cell 1

# # --- build model(s) ---
# fof_A = FunctionOnFunctionModel(pairs, criterion="A", intercept=intercept, dtype=tf.float64, response_basis=None)
# fof_D = FunctionOnFunctionModel(pairs, criterion="D", intercept=intercept, dtype=tf.float64, response_basis=None)

# # --- design coefficients Γ: (runs, Kx_total) -- same RNG/state as Cell 1 ---
# Gamma = rng.normal(size=(runs, fof_A.Kx))

# # --- shapes & quick info ---
# print("=== Function-on-Function (to match SoF) ===")
# print(f"Kx_list={fof_A.Kx_list}, sum={fof_A.Kx} | Kb_list={fof_A.Kb_list}, sum={fof_A.Kb}")
# print(f"p={fof_A.p} (should equal {p}), runs={runs}")
# print("J shape:", fof_A.J_np.shape)

# Z = fof_A.model_matrix(Gamma)
# M = fof_A.information_matrix(Z)
# print("Z shape:", Z.shape, "| M shape:", M.shape)

# # --- criteria values (should match those you copied from Cell 1) ---
# A_val = fof_A.objective_num(Gamma)
# D_val = fof_D.objective_num(Gamma)

# print("\nCompare to your SoF notes:")
# print(f"A-opt (trace(M^{-1})) loss: {A_val:.12f}")
# print(f"D-opt (-logdet M) loss:     {D_val:.12f}")


In [ ]:
# import numpy as np
# import tensorflow as tf

# from bases.bspline import BSplineBasis
# from models.sof import ScalarOnFunctionModel
# from inner_products.penalty import PenaltyBuilder

# # --- Build a simple 1-factor setup ---
# xB = BSplineBasis(degree=3, total_knots_num=12)  # input basis (Kx)
# bB = BSplineBasis(degree=3, total_knots_num=12)  # parameter basis (Kb)
# basis_pairs = [(xB, bB), (xB, bB)]

# # --- PenaltyBuilder sanity ---
# pb = PenaltyBuilder([bB], quad_points=256, diff_h=1e-4)
# R0 = pb.build()
# print("R0 shape:", R0.shape)              # expect (Kb, Kb)
# evals = np.linalg.eigvalsh((R0 + R0.T) * 0.5)
# print("R0 min eigenvalue:", evals.min())  # should be >= -1e-10 (numeric noise)

# # --- Build two SoF models (no penalty vs. λ>0) ---
# m0 = ScalarOnFunctionModel(basis_pairs, criterion="D", intercept=True, lambda_penalty=None)
# m1 = ScalarOnFunctionModel(basis_pairs, criterion="D", intercept=True, lambda_penalty=1.0)

# runs = 8
# Gamma = np.random.uniform(-1, 1, size=(runs, m0.Kx)).astype(np.float64)

# # Z and M for the base (no-penalty) model
# Z0 = m0.model_matrix(tf.constant(Gamma))
# M0 = m0.information_matrix(Z0).numpy()

# # Check that m1 builds R̃ (p x p) with zero top-left for intercept
# R_tilde = m1.R.numpy()
# print("R̃ shape:", R_tilde.shape, "p=", m1.p)
# print("Intercept penalty entry (should be 0):", R_tilde[0, 0])

# # Verify that regularization equals M + λR̃ (regularization is applied only inside objective/report)
# M_reg_expected = M0 + 1.0 * R_tilde
# M_reg_actual = m1._regularize_information(tf.constant(M0)).numpy()
# print("Regularization close:", np.allclose(M_reg_expected, M_reg_actual, atol=1e-10))

# # Quick criterion sanity (no need for equality, just that λ changes it smoothly)
# loss0 = float(m0.objective(tf.constant(Gamma)).numpy().reshape(()))
# loss1 = float(m1.objective(tf.constant(Gamma)).numpy().reshape(()))
# print("Loss (λ=None):", loss0)
# print("Loss (λ=1.0):", loss1)


In [ ]:
# import numpy as np
# from bases.polynomial import PolynomialBasis
# from inner_products.penalty import PenaltyBuilder

# # Analytic R0 for monomial basis {1, t, ..., t^d}
# def analytic_R_poly(degree: int) -> np.ndarray:
#     K = degree + 1
#     R = np.zeros((K, K), dtype=float)
#     for i in range(2, K):
#         for j in range(2, K):
#             R[i, j] = (i * (i - 1) * j * (j - 1)) / (i + j - 3)
#     return R

# # Build numeric R0
# pB = PolynomialBasis(degree=3)
# pb = PenaltyBuilder([pB], quad_points=256, diff_h=1e-4)
# R0_num = pb.build()

# R0_ref = analytic_R_poly(3)

# print("Numeric R0:\n", np.round(R0_num, 6))
# print("Analytic R0:\n", R0_ref)
# print("Abs diff:\n", np.round(np.abs(R0_num - R0_ref), 8))
# print("Max abs diff:", float(np.max(np.abs(R0_num - R0_ref))))

In [ ]:
# from bases.polynomial import PolynomialBasis
# from models.sof import ScalarOnFunctionModel
# import numpy as np, tensorflow as tf

# xB = PolynomialBasis(3)   # just to get a quick Kx
# bB = PolynomialBasis(3)
# m0 = ScalarOnFunctionModel([(xB,bB)], criterion="D", intercept=True)            # no penalty
# m1 = ScalarOnFunctionModel([(xB,bB)], criterion="D", intercept=True, lambda_penalty=0.1)

# runs = 6
# Gamma = np.random.uniform(-1,1,(runs,m0.Kx)).astype(np.float64)
# print("penalty active?", m1.R is not None, "R̃ shape:", None if m1.R is None else m1.R.shape)
# print("loss (λ=None) vs (λ=0.1):", float(m0.objective(Gamma)), float(m1.objective(Gamma)))


In [ ]:
# # --- Smoke test: Scalar-on-Function + NBDO (no penalty) ---
# import os, numpy as np, tensorflow as tf

# from bases.bspline import BSplineBasis
# from models.sof import ScalarOnFunctionModel
# from optimizers.nbdo import NBDO

# # Reproducibility
# SEED = 42
# np.random.seed(SEED)
# tf.random.set_seed(SEED)
# tf.keras.utils.set_random_seed(SEED)

# # Bases (1 factor for speed; feel free to add more)
# xB = BSplineBasis(degree=3, total_knots_num=12)   # input basis
# bB = BSplineBasis(degree=3, total_knots_num=12)   # parameter basis
# pairs = [(xB, bB)]

# # Model (no penalty)
# model = ScalarOnFunctionModel(
#     basis_pairs=pairs,
#     criterion="A",       # or "A"
#     intercept=True,
#     lambda_penalty=None  # NO PENALTY
# )

# # NBDO setup
# runs = 30
# num_designs = 256
# latent_dim = 4

# opt = NBDO(model=model, latent_dim=latent_dim, seed=SEED, verbose=False)
# opt.compute_train_set(num_designs=num_designs, runs=runs, random_state=SEED)

# # Fit the autoencoder (small epochs for quick smoke)
# opt.fit(epochs=50, batch_size=32, patience=5)

# # BO in latent space (small n_calls for speed)
# report0, design0 = opt.optimize(n_calls=10, n_random_starts=5)

# print("=== NO PENALTY ===")
# print(f"runs = {runs}, Kx = {model.Kx}, Kb = {model.Kb}, p = {model.p}")
# print(f"Optimal report (criterion='A' -> trace(M)): {report0:.6g}")
# print("Design shape:", design0.shape)

In [ ]:
# --- Smoke test: Scalar-on-Function + NBDO (no penalty) ---
import os, numpy as np, tensorflow as tf

from bases.bspline import BSplineBasis
from models.sof import ScalarOnFunctionModel
from optimizers.nbdo import NBDO

# Reproducibility
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
tf.keras.utils.set_random_seed(SEED)

# Bases (1 factor for speed; feel free to add more)
xB = BSplineBasis(degree=3, total_knots_num=12)   # input basis
bB = BSplineBasis(degree=3, total_knots_num=12)   # parameter basis
pairs = [(xB, bB)]

# Model (no penalty)
model = ScalarOnFunctionModel(
    basis_pairs=pairs,
    criterion="A",       # or "A"
    intercept=True,
    lambda_penalty=0.1  # NO PENALTY
)

# NBDO setup
runs = 30
num_designs = 256
latent_dim = 4

opt = NBDO(model=model, latent_dim=latent_dim, seed=SEED, verbose=False)
opt.compute_train_set(num_designs=num_designs, runs=runs, random_state=SEED)

# Fit the autoencoder (small epochs for quick smoke)
opt.fit(epochs=50, batch_size=32, patience=5)

# BO in latent space (small n_calls for speed)
report0, design0 = opt.optimize(n_calls=10, n_random_starts=5)

print("=== NO PENALTY ===")
print(f"runs = {runs}, Kx = {model.Kx}, Kb = {model.Kb}, p = {model.p}")
print(f"Optimal report (criterion='A' -> trace(M)): {report0:.6g}")
print("Design shape:", design0.shape)

# === NO PENALTY ===
# runs = 30, Kx = 14, Kb = 14, p = 15
# Optimal report (criterion='A' -> trace(M)): 4132.03
# Design shape: (30, 14)